##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Robust machine learning on streaming data using Kafka and Tensorflow-IO

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/io/tutorials/kafka"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/kafka.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/io/blob/master/docs/tutorials/kafka.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
      <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/io/docs/tutorials/kafka.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Caution: In addition to python packages this notebook uses `sudo apt-get install` to install third party packages.

## Overview

This tutorial focuses on streaming data from a [Kafka](https://docs.confluent.io/current/getting-started.html) cluster into a `tf.data.Dataset` which is then used in conjunction with `tf.keras` for training and inference.

Kafka is primarily a distributed event-streaming platform which provides scalable and fault-tolerant streaming data across data pipelines. It is an essential technical component of a plethora of major enterprises where mission-critical data delivery is a primary requirement.

**NOTE:** A basic understanding of the [kafka components](https://docs.confluent.io/current/kafka/introduction.html) will help you in following the tutorial with ease.

## Setup and usage

### Install the required tensorflow-io and kafka packages

In [ ]:
print("Installing the tensorflow-io package !")
!pip install tensorflow-io

print("Installing the kafka-python package !")
!pip install kafka-python

### Import packages

In [ ]:
import os
from datetime import datetime
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

### Validate tf and tfio imports

In [ ]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

### Download and setup Kafka and Zookeeper instances

For demo purposes, the following instances are setup locally:

- Kafka (Brokers: 127.0.0.1:9092)
- Zookeeper (Node: 127.0.0.1:2181)



In [ ]:
!curl -sSOL http://packages.confluent.io/archive/5.4/confluent-community-5.4.1-2.12.tar.gz
!tar -xzf confluent-community-5.4.1-2.12.tar.gz

We use the default configurations for spinning up these instances as provided by the confluent package.

In [ ]:
!cd confluent-5.4.1 && bin/zookeeper-server-start -daemon etc/kafka/zookeeper.properties
!cd confluent-5.4.1 && bin/kafka-server-start -daemon etc/kafka/server.properties
!cd confluent-5.4.1 && bin/schema-registry-start -daemon etc/schema-registry/schema-registry.properties
!echo "Waiting for 10 secs until kafka, zookeeper and schema registry services are up and running"
!sleep 10


Once the instances are started as daemon processes, we can grep for `kafka` in the processes list. The three java processes correspond to zookeeper, kafka and the schema-registry instances.

In [ ]:
!ps -ef | grep kafka

Create the kafka topics with the following specs:

- susy-train: partitions=1, replication-factor=1 
- susy-test: partitions=2, replication-factor=1 

In [ ]:
!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 1 --topic susy-train
!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 2 --topic susy-test



Describe the topic for details on the configuration

In [ ]:
!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic susy-train
!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic susy-test


The replication factor 1 indicates that the data is not being replicated. This is due to the presence of a single broker in our kafka setup.

**NOTE:** In production systems, the number of bootstrap servers can be in the range of 100's of nodes. That is where the fault-tolerance using replication comes into picture.

Please refer the [docs](https://kafka.apache.org/documentation/#replication) for more details.



### Dataset

**NOTE:** In actual scenarios, since kafka is an event streaming platform, data from various sources can be written into kafka. For instance:

- syslog records
- Web traffic logs
- Firewall logs
- Astronomical entity metrics
- IoT sensor data
- Product reviews
- Fashion images and many more.

#### Description

For the purpose of this tutorial, we will download the [SUSY](https://archive.ics.uci.edu/ml/datasets/SUSY#) dataset and feed it into kafka manually, so as to simulate a streaming environment.

Th goal of this classification problem is to distinguish between a signal process which produces supersymmetric particles and a background process which does not. In our case, we simulate a streaming enviornment where we train a model to classify these events and then infer on the test data by utilizing tensorflow-io.


In [ ]:
!curl -sSOL https://archive.ics.uci.edu/ml/machine-learning-databases/00279/SUSY.csv.gz
!gzip -d SUSY.csv.gz

#### Explore the dataset

The first column is the class label (1 for signal, 0 for background), followed by the 18 features (8 low-level features then 10 high-level features).

**NOTE:** The first 8 features are kinematic properties measured by the particle detectors in the accelerator. The last ten features are functions of the first 8 features; these are high-level features derived by physicists to help discriminate between the two classes

In [ ]:
COLUMNS = [
          #  labels
           'class',
          #  low-level features
           'lepton_1_pT',
           'lepton_1_eta',
           'lepton_1_phi',
           'lepton_2_pT',
           'lepton_2_eta',
           'lepton_2_phi',
           'missing_energy_magnitude',
           'missing_energy_phi',
          #  high-level derived features
           'MET_rel',
           'axial_MET',
           'M_R',
           'M_TR_2',
           'R',
           'MT2',
           'S_R',
           'M_Delta_R',
           'dPhi_r_b',
           'cos(theta_r1)'
           ]

In [ ]:
susy_df = pd.read_csv('SUSY.csv', header=None)
susy_df.columns=COLUMNS
susy_df.head()

In [ ]:
# total number of datapoints and columns
len(susy_df), len(susy_df.columns)

**Since the number of records are huge, let's take only a fraction of the dataset so that we spend less time on the moving the data and spend more time on understanding the functionality of the api.**

In [ ]:
df=susy_df.sample(frac=0.02, replace=True, random_state=1)
len(df), len(df[df["class"]==0]), len(df[df["class"]==1])

#### Split the dataset

We have a pretty balanced dataset for the purpose of this tutorial. We will now split the dataset into training and testing sets and store them in kafka. Thus, simulating an environment of continuous remote data retrieval for training and inference purposes.

> **NOTE:** Please do not confuse the training step with online training. It's an entirely different paradigm of training.




In [ ]:
train_df, test_df = train_test_split(df, test_size=0.4, shuffle=True)
print("Number of training samples: ",len(train_df))
print("Number of testing sample: ",len(test_df))

x_train_df = train_df.drop(["class"], axis=1)
y_train_df = train_df["class"]

x_test_df = test_df.drop(["class"], axis=1)
y_test_df = test_df["class"]

# We are splitting the data here so as to set the kafka message keys based on labels
# This helps us in storing data in multiple-partitions and retrieve the data efficiently
# using the consumer groups
x_train = list(filter(None, x_train_df.to_csv(index=False).split("\n")[1:]))
y_train = list(filter(None, y_train_df.to_csv(index=False).split("\n")[1:]))

x_test = list(filter(None, x_test_df.to_csv(index=False).split("\n")[1:]))
y_test = list(filter(None, y_test_df.to_csv(index=False).split("\n")[1:]))


In [ ]:
NUM_COLUMNS = len(x_train_df.columns)
len(x_train), len(y_train), len(x_test), len(y_test)

> #### Store the train and test data in kafka


In [ ]:


def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

write_to_kafka("susy-train", zip(x_train, y_train))
write_to_kafka("susy-test", zip(x_test, y_test))


#### Define the tfio train dataset

We will be utilizing the `IODataset` class for streaming data from kafka into tensorflow. The class inherits from `tf.data.Dataset` and thus has all the useful functionalities of `tf.data.Dataset` out of the box.


In [ ]:
def decode_kafka_item(item):
  message = tf.io.decode_csv(item.message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(item.key)
  return (message, key)

BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64
train_ds = tfio.IODataset.from_kafka('susy-train', partition=0, offset=0)
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
train_ds = train_ds.map(decode_kafka_item)
train_ds = train_ds.batch(BATCH_SIZE)

### Build and train the model

In [ ]:
# Set the parameters

OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=20


In [ ]:
# design/build the model
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(NUM_COLUMNS,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

In [ ]:
# compile the model
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [ ]:
# fit the model
model.fit(train_ds, epochs=10)

**NOTE:** Since we have used only a fraction of the dataset, our accuracy is limited to ~78% during the training phase. However, please feel free to store additional data in kafka for better a model performance. Also, since our goal was to just demonstrate the functionality of the tfio kafka datasets we have used a smaller and less-complicated neural network. However, one can increase the complexity of the model, modify the learning strategy, tune hyper-parameters etc for exploration purposes. For a baseline approach, please refer to this [article](https://www.nature.com/articles/ncomms5308#Sec11).

### Infer on the test data

For faster and scalable inference, we utilize the
`KafkaGroupIODataset` class.


In [ ]:
# dataset to stream for the susy-test topic
test_ds = tfio.experimental.streaming.KafkaGroupIODataset(
    topics=["susy-test"],
    group_id="testcg",
    servers="127.0.0.1:9092",
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
    ],
)

test_ds = test_ds.map(decode_kafka_item)
test_ds = test_ds.batch(BATCH_SIZE)

# evaluate the performance of the model on the test data
res = model.evaluate(test_ds)
print("test loss, test acc:", res)


> **NOTE:** Though this class can be used for training purposes, there are caveats which need to be addressed. Once all the messages are read from kafka and the latest offsets are committed using the `KafkaGroupIODataset`, the consumer doesn't restart reading the messages from the beginning. Thus, while training, it is possible only to train for a single epoch with the data continuously flowing in.

This kind of a functionality has limited use cases during the training phase wherein, once a datapoint has been consumed by the model it is no longer required and can be discarded.

**However, this functionality shines when it comes to robust inference with exactly-once semantics.**

#### Track the offset lag of the `testcg` consumer group w.r.t `susy-test` topic

In [ ]:
!confluent-5.4.1/bin/kafka-consumer-groups --bootstrap-server 127.0.0.1:9092 --describe --group testcg


Once the `current-offset` matches the `log-end-offset` for all the partitions, it indicates that the consumers have fetched all the messages from the kafka topic.

### References:

- Baldi, P., P. Sadowski, and D. Whiteson. “Searching for Exotic Particles in High-energy Physics with Deep Learning.” Nature Communications 5 (July 2, 2014)

- SUSY Dataset: https://archive.ics.uci.edu/ml/datasets/SUSY#

